# Create composite target size sensitive fuzzy viewshed
Uses modified Ogburn model  
https://www.researchgate.net/publication/222908522_Assessing_the_level_of_visibility_of_cultural_objects_in_past_landscapes

Builds upon Alberti's Geoprocessing implementation  
https://www.researchgate.net/publication/320490670_Fuzzy_Viewshed_ArcGIS_toolbox

History: Adam Morgan April 2021

Set up environment of the routine  
https://pro.arcgis.com/en/pro-app/latest/arcpy/classes/env.htm

In [3]:
import arcpy
import arcgis
import math
import time
from datetime import datetime
from arcpy.sa import *

# Define input datasets
Input_raster_DTM = "LakeDistrictElevation_BNG"
viewpoints_dataset = "Wainwright_Summits_BNG"

# Define object height in metres
object_target_height = 74.0

# get the extent of the DTM
desc1 = arcpy.Describe(Input_raster_DTM)
Extent= desc1.extent

# To allow overwriting outputs change overwriteOutput option to True.
arcpy.env.overwriteOutput = True
arcpy.env.scratchWorkspace=r"C:/Users/sulu3/Documents/ArcGIS/Projects/FuzzyViewshed/FuzzyViewshed.gdb"
arcpy.env.workspace=r"C:/Users/sulu3/Documents/ArcGIS/Projects/FuzzyViewshed/FuzzyViewshed.gdb"
arcpy.env.snapRaster = Input_raster_DTM
arcpy.env.extent = Extent

# Check out any necessary licenses.
arcpy.CheckOutExtension("3D")
arcpy.CheckOutExtension("spatial")
arcpy.CheckOutExtension("ImageAnalyst")

'CheckedOut'

In [4]:
# Set the Coordinate system to British National Grid
sr_BNG = arcpy.SpatialReference("British National Grid")
arcpy.env.outputCoordinateSystem = sr_BNG

Create an empty raster dataset to hold to final results  
Create the raster by copying the DTM raster, to provide the same grid extent and cell size.  
Then set all pixel values to zero to begin with.

https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/copy-raster.htm  
https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/con-.htm  
https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/delete.htm

In [5]:
def init_results_raster():
    
    print(datetime.now().strftime("%H:%M:%S") + " Initialising results raster")

    in_raster = Input_raster_DTM
    temp_rasterdataset = "temp_raster"
    config_keyword = ""
    background_value = None
    nodata_value = 0
    onebit_to_eightbit = "NONE"
    colormap_to_RGB = "NONE"
    pixel_type = ""
    scale_pixel_value = "NONE"
    RGB_to_Colormap = "NONE"
    format = ""
    transform = "NONE"
    process_as_multidimensional = "CURRENT_SLICE"
    build_multidimensional_transpose = "NO_TRANSPOSE"

    # set all pixels to zero
    global result_rasterdataset
    result_rasterdataset = "result_raster"

    if arcpy.Exists(temp_rasterdataset):
        arcpy.Delete_management(temp_rasterdataset)

    arcpy.management.CopyRaster(in_raster, temp_rasterdataset, config_keyword, background_value, nodata_value, onebit_to_eightbit, colormap_to_RGB, pixel_type, scale_pixel_value, RGB_to_Colormap, format, transform, process_as_multidimensional, build_multidimensional_transpose)

    if arcpy.Exists(result_rasterdataset):
        arcpy.Delete_management(result_rasterdataset)

    # set all cell values to 0
    result_raster = Con(IsNull(temp_rasterdataset),0.0,0.0)
    result_raster.save(result_rasterdataset)



Calculate the euclidian distance to every cell in the study area from the viewpoint.

https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/euclidean-distance.htm

In [6]:
def calc_eucl_dist():
    desc2 = arcpy.Describe(Input_raster_DTM + "/Band_1")
    Output_cell_size = desc2.meanCellWidth

    eucl_dist_name = "Eucl_Dist"
    
    if arcpy.Exists(eucl_dist_name):
        arcpy.Delete_management(eucl_dist_name)

    global eucl_dist
    eucl_dist = arcpy.sa.EucDistance(in_source_data=viewpoint, \
                                     maximum_distance=None, \
                                     cell_size=Output_cell_size, \
                                     out_direction_raster="", \
                                     distance_method="GEODESIC", \
                                     in_barrier_data="", \
                                     out_back_direction_raster="")
    eucl_dist.save(eucl_dist_name)

Use modified Ogburn equation to calculate the fuzzy membership value for each cell based on its distance from the viewpoint.  
Fuzzy membership calculation based on Alberti's implementation of Ogburn

https://www.researchgate.net/publication/320490670_Fuzzy_Viewshed_ArcGIS_toolbox

In [7]:
def calc_fuzzy_membership():

    fuzzy_memb_name = "fuzzy_memb"

    # set parameters to tune results for wind turbines
    vis_arc = 0.2
    b1 = 1000.0
    denominator = 1.8

    if arcpy.Exists(fuzzy_memb_name):
        arcpy.Delete_management(fuzzy_memb_name)

    global fuzzy_memb
    fuzzy_memb = Con(Raster("Eucl_Dist") <= float(b1), 1, 1 / (1 + 
                                            (2 * (((Raster("Eucl_Dist") - float(b1)) / 
                                            (((1 / (2 * math.tan((vis_arc * math.pi) / 
                                            180) / denominator)) * float(object_target_height)) - float(b1))))**2)))

    fuzzy_memb.save(fuzzy_memb_name)

Create a binary viewshed to show all the cells in the DTM that have line of sight to the viewpoint.

https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/viewshed-2.htm

In [8]:
def calc_binary_viewshed():
    
    in_raster = Input_raster_DTM
    in_observer_features = viewpoint
    out_agl_raster = None
    analysis_type = "FREQUENCY"
    vertical_error = "0 Meters"
    out_observer_region_relationship_table = None
    refractivity_coefficient = 0.13
    surface_offset = "0 Meters"
    observer_elevation = None
    observer_offset = 2
    inner_radius = None
    inner_radius_is_3d = "GROUND"
    outer_radius = None
    outer_radius_is_3d = "GROUND"
    horizontal_start_angle = 0
    horizontal_end_angle = 360
    vertical_upper_angle = 90
    vertical_lower_angle = -90
    analysis_method = "ALL_SIGHTLINES"

    global binary_viewshed
    binary_viewshed = "binary_vs"
        
    if arcpy.Exists(binary_viewshed):
        arcpy.Delete_management(binary_viewshed)

    arcpy.ddd.Viewshed2(in_raster, in_observer_features, binary_viewshed, out_agl_raster, \
                        analysis_type, vertical_error, out_observer_region_relationship_table, \
                        refractivity_coefficient, surface_offset, observer_elevation, observer_offset, \
                        inner_radius, inner_radius_is_3d, outer_radius, outer_radius_is_3d, horizontal_start_angle, \
                        horizontal_end_angle, vertical_upper_angle, vertical_lower_angle, analysis_method)

Multiply the binary viewshed (1 or 0) by the fuzzy membership for each cell to remove cells that cannot be seen.

https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/raster-calculator.htm

In [9]:
def calc_fuzzy_viewshed():
    
    global fuzzy_vs
    fuzzy_viewshed = "fuzzy_vs"
        
    if arcpy.Exists(fuzzy_viewshed):
        arcpy.Delete_management(fuzzy_viewshed)
        
    fuzzy_vs = binary_viewshed * fuzzy_memb
    fuzzy_vs = Con(IsNull(fuzzy_vs),0,fuzzy_vs)

    fuzzy_vs.save(fuzzy_viewshed)

https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/rename.htm

In [10]:
def calc_comp_fuzzy_viewshed():
    # Add current fuzzy viewshed to final results raster
    
    if arcpy.Exists("temp_result"):
        arcpy.Delete_management("temp_result")
    
    temp_raster = RasterCalculator(["fuzzy_vs","result_raster"],["F","R"],"R+F") 
    temp_raster.save("temp_result")
    
    # Difficult to delete sometimes
    if arcpy.Exists("result_raster"):
        #print("deleting")
        arcpy.Delete_management("result_raster")
        time.sleep(3)
        if arcpy.Exists("result_raster"):
            #print("deleting")
            arcpy.Delete_management("result_raster")
            time.sleep(3)
        
    arcpy.Rename_management("temp_result","result_raster")
    

Use the CellStatistics tool to compare the equivalent cells in two rasters and retain the highest value of each cell pair

https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/cell-statistics.htm

In [11]:
def calc_max_fuzzy_viewshed():
    # Retain max value of each cell to final results raster
    
    if arcpy.Exists("temp_result"):
        arcpy.Delete_management("temp_result")
       
    #temp_raster = np.maximum("result_raster", "fuzzy_vs)
    temp_raster = CellStatistics(["result_raster", "fuzzy_vs"], "MAXIMUM")
    temp_raster.save("temp_result")
        
    if arcpy.Exists("result_raster"):
        #print("deleting")
        arcpy.Delete_management("result_raster")
        time.sleep(3)
        if arcpy.Exists("result_raster"):
            #print("deleting")
            arcpy.Delete_management("result_raster")
            time.sleep(3)
        
    arcpy.Rename_management("temp_result","result_raster")

Routine to create fuzzy view for a single summit.  
Uncomment the summit to use based on its ObjectID.

https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/make-feature-layer.htm

In [12]:
def single_summit():
    
    # Uncomment summit ObjectID to run
    viewpoint_oid = 178 #Skiddaw
    #viewpoint_oid = 53  #Fellbarrow
    #viewpoint_oid = 18  #Blake Fell
    
    print(datetime.now().strftime("%H:%M:%S") + " Starting")
    
    global viewpoint
    viewpoint = "viewpoint"
        
    qry_string = "OBJECTID = " + str(viewpoint_oid)
    arcpy.MakeFeatureLayer_management(viewpoints_dataset, viewpoint, qry_string)

    calc_eucl_dist()
    calc_fuzzy_membership()
    calc_binary_viewshed()
    calc_fuzzy_viewshed()
                
    print(datetime.now().strftime("%H:%M:%S") + " Done.")
    

Define routine to loop through all the Wainwrights to create a single composite fuzzy viewshed.  
Note - Uncomment the composite routine needed ie max fuzzy values or summed fuzzy values for each cell.

https://pro.arcgis.com/en/pro-app/latest/arcpy/data-access/searchcursor-class.htm  
https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/calculate-statistics.htm  
https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/build-pyramids.htm

In [13]:
def all_summits():
    
    global viewpoint
    viewpoint = "viewpoint"
    
    print(datetime.now().strftime("%H:%M:%S") + " Starting")
    
    # Set up an empty raster to hold the composite results
    init_results_raster()
    
    # Loop through every Wainwright in the viewpoints dataset
    with arcpy.da.SearchCursor(viewpoints_dataset,['OID@','Name']) as cursor:
        for row in cursor:
            viewpoint_oid = row[0]
            
            if viewpoint_oid > 0:
                viewpoint_name = str(row[1])
                    
                print(datetime.now().strftime("%H:%M:%S") + " Creating viewshed for " + str(viewpoint_oid) + ": " + viewpoint_name)
         
                qry_string = "OBJECTID = " + str(viewpoint_oid)
                arcpy.MakeFeatureLayer_management(viewpoints_dataset, viewpoint, qry_string)

                calc_eucl_dist()
                calc_fuzzy_membership()
                calc_binary_viewshed()
                calc_fuzzy_viewshed()

                # Uncomment the composite routine needed ie max fuzzy values or summed fuzzy values for each cell
                calc_max_fuzzy_viewshed()
                #calc_comp_fuzzy_viewshed()

    # Calc statistics to speed up drawing on map 
    arcpy.management.CalculateStatistics("result_raster")
    arcpy.management.BuildPyramids("result_raster")
    
    print(datetime.now().strftime("%H:%M:%S") + " Done.")
    

Finally, once all the routines have been defined, run the program.  
Uncomment the routine to run - single wainwright, or all wainwrights.

In [14]:
single_summit()
#all_summits()

22:33:13 Starting
22:38:43 Done.
